# Oscilador Híbrido Cuántico: Formalismo de Koopman-van Hove

Este cuaderno explora la dinámica de un sistema híbrido clásico-cuántico utilizando el formalismo de Koopman-van Hove (KvH), que depende de un parámetro de deformación $\kappa$. Se analizan los límites clásico y cuántico, la retro-reacción, el entrelazamiento y la representación de los estados en el espacio de fases.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

from scipy.linalg import expm
from scipy.special import hermite, factorial
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from tqdm.notebook import tqdm

from fig_config import add_grid, figure_features
figure_features()


# 1. Definiciones y Funciones del Modelo

In [ ]:
# --- FUNCIONES DE FIGURAS ---
def save_animation_with_timestamp(animation_object, base_filename, potential, coupling, output_dir="Output", **savefig_kwargs):
    """
    Guarda una animación de Matplotlib en un directorio específico con una marca de tiempo y metadatos.
    """
    if SAVE_ANIMS:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.datetime.now().strftime("_%Y_%m_%d_%H_%M_%S")
        
        descriptive_name = f"{base_filename}_{potential}_{coupling}{timestamp}.gif"
        final_filepath = os.path.join(output_dir, descriptive_name)
        
        print(f"Guardando animación en: {final_filepath}")
        if 'progress_callback' not in savefig_kwargs:
            savefig_kwargs['progress_callback'] = lambda i, n: print(f'Guardando frame {i+1}/{n}', end='\r')
        
        animation_object.save(final_filepath, **savefig_kwargs)
        print(f"\nAnimación guardada con éxito en '{final_filepath}'")
    else:
        print("Guardado de animaciones desactivado.")

def save_figure_with_timestamp(base_filename, potential, coupling, output_dir="Output", **savefig_kwargs):
    """
    Guarda la figura actual de Matplotlib en un directorio específico con una marca de tiempo y metadatos.
    """
    if SAVE_FIGS:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.datetime.now().strftime("_%Y_%m_%d_%H_%M_%S")

        descriptive_name = f"{base_filename}_{potential}_{coupling}{timestamp}.pdf"
        final_filepath = os.path.join(output_dir, descriptive_name)
        
        print(f"Guardando figura en: {final_filepath}")
        plt.savefig(final_filepath, **savefig_kwargs)
        print("Figura guardada.")
    else:
        print("Guardado de figuras desactivado.")

# --- FUNCIONES GENERALES ---
def normalize(vec):
    norm = np.linalg.norm(vec)
    if norm < 1e-15: return vec
    return vec / norm

def expected_value(psi_vector, Operator_matrix):
    if psi_vector.ndim == 1:
        return np.conjugate(psi_vector) @ Operator_matrix @ psi_vector
    else:
        raise ValueError("psi_vector debe ser 1D.")

def prob_density_coeffs(psi_coeffs_vector):
    return np.abs(psi_coeffs_vector)**2

# --- FUNCIONES DEL MODELO MIXTO ---
def U_classical(x, potential_type='harmonic', m=1.0, omega=1.0, c4=0.0):
    if potential_type == 'harmonic':
        return 0.5 * m * omega**2 * (x @ x)
    elif potential_type == 'quartic':
        return 0.5 * m * omega**2 * (x @ x) + c4 * (x @ x @ x @ x)
    else:
        raise ValueError("Tipo de potencial no reconocido.")

def L_h_generator(x_op, p_op, lambda_x_op, lambda_p_op, m_h, omega_h, hbar, kappa, potential_type='harmonic', c4=0.0):
    if kappa == 0:
        U_prime = m_h * omega_h**2 * x_op
        if potential_type == 'quartic':
            U_prime += 4 * c4 * (x_op @ x_op @ x_op)
        return (p_op / m_h) @ lambda_x_op - U_prime @ lambda_p_op
    else:
        term1 = x_op - 0.5 * hbar * kappa * lambda_p_op
        term2 = x_op + 0.5 * hbar * kappa * lambda_p_op
        U1 = U_classical(term1, potential_type, m_h, omega_h, c4)
        U2 = U_classical(term2, potential_type, m_h, omega_h, c4)
        return (p_op / m_h) @ lambda_x_op + (1/kappa) * (U1 - U2)

def create_unified_coupling_term(Xh, Ph, Lxh, Lph, Xq, Pq, hbar, kappa, epsilon, coupling_type='x_mix_x'):
    """Crea el término de acoplo usando los operadores mixtos dependientes de kappa."""
    X_mix = Xh - (0.5 * hbar * kappa * Lph)
    P_mix = Ph + (0.5 * hbar * kappa * Lxh)
    
    if coupling_type == 'x_mix_x':
        return epsilon * np.kron(X_mix, Xq)
    elif coupling_type == 'p_mix_p':
        return epsilon * np.kron(P_mix, Pq)
    elif coupling_type == 'x_mix2_x2':
        return epsilon * np.kron(X_mix @ X_mix, Xq @ Xq)
    elif coupling_type == 'p_mix2_p2':
        return epsilon * np.kron(P_mix @ P_mix, Pq @ Pq)
    elif coupling_type == 'x_mix_p':
        return epsilon * np.kron(X_mix, Pq)
    else:
        raise ValueError("Tipo de acoplo no reconocido.")

def joint_evolution_wrapper(psi0, H_eff, t, hbar=1.0):
    """Evoluciona un estado híbrido."""
    return expm(-1j * t * H_eff / hbar) @ psi0

def number_state_wavefunction(n, x, m=1.0, omega=1.0, hbar=1.0):
    """Calcula la función de onda normalizada ψ_n(x) para el autoestado |n>."""
    alpha = np.sqrt(m * omega / hbar)
    norm_factor = (alpha / np.sqrt(np.pi))**0.5 / np.sqrt(2**n * factorial(n, exact=True))
    hermite_poly = hermite(n)
    return norm_factor * np.exp(-0.5 * (alpha * x)**2) * hermite_poly(alpha * x)

def reconstruct_husimi_from_rho_c(rho_c_matrix, x_grid, p_grid, N_max_h):
    """
    Reconstruye la función Q de Husimi a partir de la matriz densidad reducida clásica.
    Esto proporciona una verdadera densidad de probabilidad en el espacio de fases.
    """
    nx, ny = len(x_grid), len(p_grid)
    husimi_q = np.zeros((nx, ny))
    
    for i in range(nx):
        for j in range(ny):
            x = x_grid[i]
            p = p_grid[j]
            
            alpha_x = x / np.sqrt(2.0)
            alpha_p = p / np.sqrt(2.0)
            
            coherent_x_coeffs = coherent_state_coeffs(alpha_x, N_max_h)
            coherent_p_coeffs = coherent_state_coeffs(alpha_p, N_max_h)
            
            coherent_state_vec = np.kron(coherent_x_coeffs, coherent_p_coeffs)
            
            coherent_state_vec = normalize(coherent_state_vec)
            
            val = np.conjugate(coherent_state_vec) @ rho_c_matrix @ coherent_state_vec
            husimi_q[i, j] = np.real(val) / np.pi
            
    return husimi_q

# --- FUNCIONES PARA ESTADOS INICIALES ESTRUCTURADOS ---
def coherent_state_coeffs(alpha, N_max):
    """Genera los coeficientes de un estado coherente en la base de Fock."""
    dim = N_max + 1
    coeffs = np.zeros(dim, dtype=np.complex128)
    if np.abs(alpha) < 1e-9: # Maneja el caso alpha=0, que es el estado |0>
        coeffs[0] = 1.0
        return coeffs
    norm_sq = np.abs(alpha)**2
    for n in range(dim):
        coeffs[n] = np.exp(-norm_sq / 2) * (alpha**n) / np.sqrt(factorial(n, exact=True))
    return normalize(coeffs)

def gaussian_fock_state_coeffs(n0, sigma, N_max):
    """
    Genera coeficientes para un estado con una distribución gaussiana
    en la base de autoestados del operador número |n>.
    n0: centro de la gaussiana (puede ser no entero).
    sigma: anchura de la gaussiana.
    """
    dim = N_max + 1
    n_values = np.arange(dim)
    coeffs = np.exp(-(n_values - n0)**2 / (2 * sigma**2))
    return normalize(coeffs.astype(np.complex128))

def twin_coherent_state_coeffs(alpha, N_max):
    """
    Genera los coeficientes de un estado par (|alpha> + |-alpha>),
    que es una superposición de dos estados coherentes.
    """
    coeffs_plus = coherent_state_coeffs(alpha, N_max)
    coeffs_minus = coherent_state_coeffs(-alpha, N_max)
    return normalize(coeffs_plus + coeffs_minus)

# 2. Configuración del Sistema, Operadores y Estado Inicial

In [ ]:
# --- PARÁMETROS ---
m_h, m_q, omega_h, omega_q, hbar = 1.0, 1.0, 1.0, 1.0, 1.0
kappa_default, epsilon, C4 = 0.5, 0.1, 0.1
POTENTIAL_TYPE, COUPLING_TYPE = 'harmonic', 'x_mix_p' # Harmonic/Quartic, x_mix_x/p_mix_p/x_mix2_x2/p_mix2_p2/x_mix_p
SAVE_FIGS = False
SAVE_ANIMS = False
N_max_h, N_max_q = 5, 5

# --- DIMENSIONES ---
dim_fock_h_single = N_max_h + 1
dim_H_hybrid = dim_fock_h_single ** 2
dim_fock_q = N_max_q + 1
times = np.linspace(0, 10 * np.pi, 200)

# --- OPERADORES ---
id_h_single = np.identity(dim_fock_h_single); id_q = np.identity(dim_fock_q)
a_h_single = np.diag([np.sqrt(i) for i in range(1, dim_fock_h_single)], k=1); aPlus_h_single = a_h_single.T
a_q = np.diag([np.sqrt(i) for i in range(1, dim_fock_q)], k=1); aPlus_q = a_q.T
a_xh = np.kron(a_h_single, id_h_single); aPlus_xh = np.kron(aPlus_h_single, id_h_single)
a_ph = np.kron(id_h_single, a_h_single); aPlus_ph = np.kron(id_h_single, aPlus_h_single)
Xh_op = np.sqrt(hbar/(2*m_h*omega_h)) * (aPlus_xh + a_xh)
Lambda_xh_op = 1j * np.sqrt((hbar*m_h*omega_h)/2) * (aPlus_xh - a_xh)
Ph_op = np.sqrt(hbar*m_h*omega_h/2) * (aPlus_ph + a_ph)
Lambda_ph_op = 1j * np.sqrt((hbar*m_h*omega_h)/2) * (aPlus_ph - a_ph)
X_q_op = np.sqrt(hbar/(2*m_q*omega_q)) * (aPlus_q + a_q)
P_q_op = 1j * np.sqrt(hbar*m_q*omega_q/2) * (aPlus_q - a_q)
H_q_op = hbar * omega_q * (aPlus_q @ a_q + 0.5 * id_q)
X_mix_op = Xh_op - (0.5 * hbar * kappa_default * Lambda_ph_op)
P_mix_op = Ph_op + (0.5 * hbar * kappa_default * Lambda_xh_op)


Xh_joint_op = np.kron(Xh_op, id_q)
Ph_joint_op = np.kron(Ph_op, id_q)
X_mix_joint_op = np.kron(X_mix_op, id_q)
P_mix_joint_op = np.kron(P_mix_op, id_q)
Xq_joint_op = np.kron(np.identity(dim_H_hybrid), X_q_op)
Pq_joint_op = np.kron(np.identity(dim_H_hybrid), P_q_op)


# --- ESTADO INICIAL ---
# 1. Subsistema Clásico:
#    Crea dos paquetes de onda distintos, por ejemplo, en x= +/- x0_c.
x0_c = 1.5
p0_c = 0.5  
alpha_x_cat = x0_c / np.sqrt(2.0)
alpha_p_cat = p0_c / np.sqrt(2.0)

psi0_c_x_coeffs = twin_coherent_state_coeffs(alpha_x_cat, N_max_h)
# Para 'p', usamos un estado coherente simple para mantener los paquetes localizados en momento.
psi0_c_p_coeffs = coherent_state_coeffs(alpha_p_cat, N_max_h)
psi0_c_coeffs = normalize(np.kron(psi0_c_x_coeffs, psi0_c_p_coeffs))
psi0_c_coeffs[0] /= 2
psi0_c_coeffs = normalize(psi0_c_coeffs)

# 2. Subsistema Cuántico: Estado con distribución gaussiana en la base de Fock.
#    Centrado en n=2 con una anchura de 1.5.
n0_q = 2.0
sigma_q = 1.5
psi0_q_coeffs = gaussian_fock_state_coeffs(n0_q, sigma_q, N_max_q)

# 3. Estado Híbrido Conjunto
psi0_joint = normalize(np.kron(psi0_c_coeffs, psi0_q_coeffs))

# --- ESTADO INICIAL COMPLETAMENTE ALEATORIO: Ahora mismo desactivado. ---
# np.random.seed(42)
# psi0_c_coeffs = normalize(np.random.rand(dim_H_hybrid) + 1j*np.random.rand(dim_H_hybrid))
# psi0_q_coeffs = normalize(np.random.rand(dim_fock_q) + 1j*np.random.rand(dim_fock_q))
# psi0_joint = normalize(np.kron(psi0_c_coeffs, psi0_q_coeffs))

# --- HAMILTONIANO PARA ANÁLISIS BÁSICO (CON kappa_default) ---
L_h_ham_default = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_default, POTENTIAL_TYPE, C4)
H_eff_sin_acoplo_default = np.kron(L_h_ham_default, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op)
H_acoplo_op_default = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_default, epsilon, COUPLING_TYPE)
H_eff_con_acoplo_default = H_eff_sin_acoplo_default + H_acoplo_op_default

# 3. Análisis Fundamental: Valores Esperados y Conservación de la Norma

In [ ]:
# --- Celda de Análisis: Comparación de Valores Esperados y Norma para Diferentes Acoplos ---

# --- PARÁMETROS ---
kappa_analysis = kappa_default # Usamos el kappa por defecto definido en la configuración
epsilon_analysis = 0.1 # Fuerza del acoplo para este análisis

# --- CONSTRUCCIÓN DE LOS HAMILTONIANOS ---
# 1. Generador de base (sin acoplo)
L_h_base = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_analysis, POTENTIAL_TYPE, C4)
H_eff_no_coupling = np.kron(L_h_base, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op)

# 2. Hamiltoniano con acoplo x_mix_x
H_acoplo_xmix_x = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_analysis, epsilon_analysis, 'x_mix_x')
H_eff_xmix_x = H_eff_no_coupling + H_acoplo_xmix_x

# 3. Hamiltoniano con acoplo x_mix_p
H_acoplo_xmix_p = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_analysis, epsilon_analysis, 'x_mix_p')
H_eff_xmix_p = H_eff_no_coupling + H_acoplo_xmix_p


# --- DICCIONARIO PARA ALMACENAR SERIES DE DATOS ---
data_series = {
    'no_coupling': {},
    'x_mix_x': {},
    'x_mix_p': {}
}

# --- CÁLCULO ---
hamiltonians = {
    'no_coupling': H_eff_no_coupling,
    'x_mix_x': H_eff_xmix_x,
    'x_mix_p': H_eff_xmix_p
}

for case, H_eff in hamiltonians.items():
    print(f"Calculando evolución para el caso: {case}...")
    
    for obs in ['x_mix', 'p_mix', 'x_q', 'p_q', 'norm']:
        data_series[case][obs] = []
        
    for t_val in tqdm(times):
        psi_t = joint_evolution_wrapper(psi0_joint, H_eff, t_val)
        
        # Operadores mixtos (dependen de kappa)
        X_mix = Xh_op - (0.5 * hbar * kappa_analysis * Lambda_ph_op)
        P_mix = Ph_op + (0.5 * hbar * kappa_analysis * Lambda_xh_op)

        X_mix_joint = np.kron(X_mix, id_q)
        P_mix_joint = np.kron(P_mix, id_q)
        
        data_series[case]['x_mix'].append(np.real(expected_value(psi_t, X_mix_joint)))
        data_series[case]['p_mix'].append(np.real(expected_value(psi_t, P_mix_joint)))
        data_series[case]['x_q'].append(np.real(expected_value(psi_t, Xq_joint_op)))
        data_series[case]['p_q'].append(np.real(expected_value(psi_t, Pq_joint_op)))
        data_series[case]['norm'].append(np.linalg.norm(psi_t))
        
# --- PLOTEO Y GUARDADO ---
plot_info = {
    'x_mix': {'label': r'$\langle \hat{X}_{mix} \rangle$', 'title': fr'Posición Mixta Esperada $(\kappa = {kappa_analysis})$'},
    'p_mix': {'label': r'$\langle \hat{P}_{mix} \rangle$', 'title': fr'Momento Mixto Esperado $(\kappa = {kappa_analysis})$'},
    'x_q':   {'label': r'$\langle \hat{X}_Q \rangle$', 'title': 'Posición Cuántica Esperada'},
    'p_q':   {'label': r'$\langle \hat{P}_Q \rangle$', 'title': 'Momento Cuántico Esperado'},
    'norm': {'label': r'Norma $||\Psi_H||$', 'title': 'Conservación de la Norma'}
}

print("\nGenerando y guardando gráficos...")
for key, info in plot_info.items():
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(times, data_series['no_coupling'][key], label='Sin Acoplo', color='black')
    ax.plot(times, data_series['x_mix_x'][key], label='Acoplo $X_{mix} \\otimes X_q$', color='navy', ls = '--')
    ax.plot(times, data_series['x_mix_p'][key], label='Acoplo $X_{mix} \\otimes P_q$', color='red', ls = '--')
    
    ax.set_title(info['title'])
    ax.set_ylabel(info['label'])
    ax.set_xlabel('Tiempo (u.a.)')
    if key == 'norm':
        ax.set_ylim(0.99, 1.01)
    
    ax.legend()
    add_grid(ax)
    
    base_filename = f"expectation_values_evolution_{key}"
    save_figure_with_timestamp(base_filename, POTENTIAL_TYPE, f"multi_coupling_k{kappa_analysis}", bbox_inches='tight')
    plt.show()

In [ ]:
norm_no_acoplo_list, ex_xmix_no_acoplo_list, ex_pmix_no_acoplo_list, ex_xq_no_acoplo_list, ex_pq_no_acoplo_list = [], [], [], [], []
norm_con_acoplo_list, ex_xmix_con_acoplo_list, ex_pmix_con_acoplo_list, ex_xq_con_acoplo_list, ex_pq_con_acoplo_list = [], [], [], [], []

print("Calculando evolución sin acoplo...")
for t_val in tqdm(times):
    psi_t_h = joint_evolution_wrapper(psi0_joint, H_eff_sin_acoplo_default, t_val)
    norm_no_acoplo_list.append(np.linalg.norm(psi_t_h))
    ex_xmix_no_acoplo_list.append(np.real(expected_value(psi_t_h, X_mix_joint_op)))
    ex_pmix_no_acoplo_list.append(np.real(expected_value(psi_t_h, P_mix_joint_op)))
    ex_xq_no_acoplo_list.append(np.real(expected_value(psi_t_h, Xq_joint_op)))
    ex_pq_no_acoplo_list.append(np.real(expected_value(psi_t_h, Pq_joint_op)))

print(f"Calculando evolución con acoplo (epsilon = {epsilon})...")
for t_val in tqdm(times):
    psi_t_h = joint_evolution_wrapper(psi0_joint, H_eff_con_acoplo_default, t_val)
    norm_con_acoplo_list.append(np.linalg.norm(psi_t_h))
    ex_xmix_con_acoplo_list.append(np.real(expected_value(psi_t_h, X_mix_joint_op)))
    ex_pmix_con_acoplo_list.append(np.real(expected_value(psi_t_h, P_mix_joint_op)))
    ex_xq_con_acoplo_list.append(np.real(expected_value(psi_t_h, Xq_joint_op)))
    ex_pq_con_acoplo_list.append(np.real(expected_value(psi_t_h, Pq_joint_op)))

fig_exp_joint, axs = plt.subplots(3, 2, figsize=(15, 15), sharex='col')
axs[0,0].plot(times, ex_xmix_no_acoplo_list, label='Sin Acoplo', color = "navy"); axs[0,0].plot(times, ex_xmix_con_acoplo_list, label=f'Con Acoplo ($\\varepsilon$={epsilon})', linestyle='--', color = "red"); axs[0,0].set_ylabel(r'$\langle \hat{X}_{mix} \rangle$'); axs[0,0].set_title(fr'Posición Mixta Esperada $(\kappa = {kappa_default})$'); axs[0,0].legend()
axs[1,0].plot(times, ex_pmix_no_acoplo_list, label='Sin Acoplo', color = "navy"); axs[1,0].plot(times, ex_pmix_con_acoplo_list, label=f'Con Acoplo ($\\varepsilon$={epsilon})', linestyle='--', color = "red"); axs[1,0].set_ylabel(r'$\langle \hat{P}_{mix} \rangle$'); axs[1,0].set_title(fr'Momento Mixta Esperado $(\kappa = {kappa_default})$'); axs[1,0].legend()
axs[0,1].plot(times, ex_xq_no_acoplo_list, label='Sin Acoplo', color = "navy"); axs[0,1].plot(times, ex_xq_con_acoplo_list, label=f'Con Acoplo ($\\varepsilon$={epsilon})', linestyle='--', color = "red"); axs[0,1].set_ylabel(r'$\langle \hat{X}_Q \rangle$'); axs[0,1].set_title('Posición Cuántica Esperada'); axs[0,1].legend()
axs[1,1].plot(times, ex_pq_no_acoplo_list, label='Sin Acoplo', color = "navy"); axs[1,1].plot(times, ex_pq_con_acoplo_list, label=f'Con Acoplo ($\\varepsilon$={epsilon})', linestyle='--', color = "red"); axs[1,1].set_ylabel(r'$\langle \hat{P}_Q \rangle$'); axs[1,1].set_title('Momento Cuántico Esperado'); axs[1,1].legend()
axs[2,0].plot(times, norm_no_acoplo_list, label='Norma Sin Acoplo'); axs[2,0].plot(times, norm_con_acoplo_list, label=f'Norma Con Acoplo ($\\varepsilon$={epsilon})', linestyle='--', color = "red"); axs[2,0].set_ylabel(r'Norma $||\Psi_H||$'); axs[2,0].set_xlabel('Tiempo (u.a.)'); axs[2,0].set_title('Conservación de la Norma'); axs[2,0].legend(); axs[2,0].set_ylim(0.99, 1.01)
axs[2,1].set_visible(False)
plt.tight_layout()
save_figure_with_timestamp("expectation_values_evolution", POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches='tight')
plt.show()

# 4. Análisis del Parámetro de Deformación `κ`

In [ ]:
# --- EQUIVALENCIA DEL MODELO SIN ACOPLO (HAY QUE USAR POTENCIAL CUADRÁTICO) ---
kappa_values_to_test = [1.0, 0.1, 0.01, 1e-4, 0.0]
results_xmix, results_pmix = {}, {}

H_acoplo = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_default, 0.0, COUPLING_TYPE)

print("Analizando convergencia para kappa -> 0...")
for kappa_val in tqdm(kappa_values_to_test, desc="Probando kappas para convergencia"):
    L_h_ham_kappa = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_eff = np.kron(L_h_ham_kappa, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op) + H_acoplo
    
    ex_xmix_list, ex_pmix_list = [], []
    for t_val in times:
        psi_t_h = joint_evolution_wrapper(psi0_joint, H_eff, t_val)
        ex_xmix_list.append(np.real(expected_value(psi_t_h, X_mix_joint_op)))
        ex_pmix_list.append(np.real(expected_value(psi_t_h, P_mix_joint_op)))
    
    results_xmix[kappa_val] = ex_xmix_list
    results_pmix[kappa_val] = ex_pmix_list

fig_conv, axs_conv = plt.subplots(1, 2, figsize=(18, 7), sharey=True)
fig_conv.suptitle(f'Equivalencia del Modelo Mixto al Clásico para potencial armónico')
ax = axs_conv[0]; ax.set_title(r'Convergencia de $\langle X_{mix} \rangle$'); ax.set_xlabel('Tiempo (u.a.)'); ax.set_ylabel('Valor esperado')
for k, d in results_xmix.items(): ax.plot(times, d, label=f'$\\kappa={k}$' if k>0 else 'KvN ($\\kappa=0$)', linestyle='--' if k==0 else '-')
ax.legend()
add_grid(ax)

ax = axs_conv[1]
ax.set_title(r'Convergencia de $\langle P_{mix} \rangle$')
ax.set_xlabel('Tiempo (u.a.)')
for k, d in results_pmix.items(): ax.plot(times, d, label=f'$\\kappa={k}$' if k>0 else 'KvN ($\\kappa=0$)', linestyle='--' if k==0 else '-')
ax.legend()
add_grid(ax)

plt.tight_layout(rect=[0, 0, 1, 0.96])
save_figure_with_timestamp("kappa_convergence", POTENTIAL_TYPE, "no_coupling", bbox_inches='tight')
plt.show()

In [ ]:
# --- BACK-REACTION ---
kappa_values_br = np.logspace(0, -4, 10)
kappa_values_br = np.append(kappa_values_br, 0)

backreaction_mae_xc = [] # Mean Absolute Error
backreaction_rmse_xc = [] # Root Mean Square Error

print("Analizando retro-reacción vs kappa con métricas robustas...")
for kappa_val in tqdm(kappa_values_br, desc="Probando kappas para back-reaction"):

    # Construir el Hamiltoniano efectivo para el valor actual de kappa
    L_h_ham_kappa = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_acoplo = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_val, epsilon, COUPLING_TYPE)
    H_eff = np.kron(L_h_ham_kappa, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op) + H_acoplo
    
    # Generar dos estados cuánticos iniciales distintos para el mismo estado clásico
    np.random.seed(123)
    psi0_c = normalize(np.random.rand(dim_H_hybrid) + 1j * np.random.rand(dim_H_hybrid))
    np.random.seed(21)
    psi0_q1 = normalize(np.random.rand(dim_fock_q) + 1j * np.random.rand(dim_fock_q))
    np.random.seed(12)
    psi0_q2 = normalize(np.random.rand(dim_fock_q) + 1j * np.random.rand(dim_fock_q))
    
    psi0_h1 = normalize(np.kron(psi0_c, psi0_q1))
    psi0_h2 = normalize(np.kron(psi0_c, psi0_q2))

    # Calcular la evolución para ambos estados iniciales
    ex_xc_1 = [np.real(expected_value(joint_evolution_wrapper(psi0_h1, H_eff, t_val), Xh_joint_op)) for t_val in times]
    ex_xc_2 = [np.real(expected_value(joint_evolution_wrapper(psi0_h2, H_eff, t_val), Xh_joint_op)) for t_val in times]
    
    # Calcular el vector de diferencia entre las dos trayectorias
    difference_vector = np.array(ex_xc_1) - np.array(ex_xc_2)
    
    # Métrica 1: Promedio de la Diferencia Absoluta (MAE)
    mae = np.mean(np.abs(difference_vector))
    backreaction_mae_xc.append(mae)
    
    # Métrica 2: Raíz del Error Cuadrático Medio (RMSE)
    rmse = np.sqrt(np.mean(np.square(difference_vector)))
    backreaction_rmse_xc.append(rmse)

fig_br_robust, axs = plt.subplots(1, 2, figsize=(18, 7), sharex=True)
fig_br_robust.suptitle('Magnitud de la Retro-reacción (Métricas Robustas) en función de $\\kappa$', fontsize=16)

axs[0].plot(kappa_values_br, backreaction_mae_xc, 'o-', label='MAE')
axs[0].set_title('Promedio de la Diferencia Absoluta (MAE)')
axs[0].set_xlabel('$\\kappa$')
axs[0].set_ylabel('Promedio$|\\langle X_C \\rangle_1 - \\langle X_C \\rangle_2|$')
axs[0].set_xscale('log')
axs[0].invert_xaxis()
add_grid(axs[0])

axs[1].plot(kappa_values_br, backreaction_rmse_xc, 'o-', color='C1', label='RMSE')
axs[1].set_title('Raíz del Error Cuadrático Medio (RMSE)')
axs[1].set_xlabel('$\\kappa$')
axs[1].set_ylabel('RMSE$(\\langle X_C \\rangle_1, \\langle X_C \\rangle_2)$')
add_grid(axs[1])

plt.tight_layout(rect=[0, 0, 1, 0.95])
save_figure_with_timestamp("backreaction_metrics", POTENTIAL_TYPE, COUPLING_TYPE,  bbox_inches='tight')
plt.show()

In [ ]:
# --- CONVERGENCIA DEL ESPECTRO ---
kappa_values_spec = np.logspace(0, -8, 20)
eigenvalues_kvh = []

print("Calculando convergencia del espectro...")

for kappa_val in tqdm(kappa_values_spec, desc="Calculando espectros"):
    L_h_ham_kappa = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    eigenvalues_kvh.append(np.sort_complex(np.linalg.eigvals(L_h_ham_kappa)))
    
eigenvalues_kvh = np.array(eigenvalues_kvh)

fig_spec, axs_spec = plt.subplots(1, 2, figsize=(18, 7), sharey=True)
fig_spec.suptitle('Equivalencia (o no) del espectro Híbrido al Clásico-Cuántico para potencial {POTENTIAL_TYPE}')
ax_real = axs_spec[0]; ax_imag = axs_spec[1]

for i in range(eigenvalues_kvh.shape[1]):
    ax_real.plot(kappa_values_spec, np.real(eigenvalues_kvh[:, i]), '-')
    ax_imag.plot(kappa_values_spec, np.imag(eigenvalues_kvh[:, i]), '-')
    
ax_real.set_xscale('log')
ax_real.invert_xaxis()
ax_real.set_title('Parte Real de los Autovalores')
ax_real.set_xlabel('$\\kappa$')
ax_real.set_ylabel('Re($\\lambda$)'); add_grid(ax_real)
ax_imag.set_xscale('log')
ax_imag.invert_xaxis()
ax_imag.set_title('Parte Imaginaria de los Autovalores')
ax_imag.set_xlabel('$\\kappa$')
add_grid(ax_imag)

save_figure_with_timestamp("spectrum_convergence", POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches='tight')
plt.show()

# 5. Análisis Avanzado de la Dinámica Híbrida

In [ ]:
# --- GRÁFICO 3D DE COVARIANZA Y OTROS OBSERVABLES ---


# --- Parámetros del Barrido ---
kappa_grid_3d = np.linspace(0, 1.0, 11)
time_grid_3d = np.linspace(0, 4 * np.pi, 100)
epsilon_3d = 0.2

# --- Definición de Observables ---
# Los nombres de archivo se generan a partir de esto
observables_to_plot_3d = {
    'xc': r'$\langle \hat{X}_C \rangle$',
    'pc': r'$\langle \hat{P}_C \rangle$',
    'xq': r'$\langle \hat{X}_Q \rangle$',
    'pq': r'$\langle \hat{P}_Q \rangle$',
    'comm': r'$Im(\langle [\hat{X}_{mix}, \hat{P}_{mix}] \rangle)$',
    'covv': r'$Cov(\hat{X}_{mix}, \hat{X}_Q)$',
    'xmix': r'$\langle \hat{X}_{mix} \rangle$',
    'pmix': r'$\langle \hat{P}_{mix} \rangle$'
}

# --- Almacenamiento de Resultados ---
results_3d = {key: np.zeros((len(kappa_grid_3d), len(time_grid_3d))) for key in observables_to_plot_3d.keys()}

# --- Estado Inicial ---
psi0_joint_3d = psi0_joint # Usamos el estado inicial ya definido

# --- Bucle Principal de Cálculo ---
print("Generando superficies de valores esperados y covarianza...")
for i, kappa_val in enumerate(tqdm(kappa_grid_3d, desc="Barriendo kappa")):
    
    # Construir Hamiltoniano y operadores dependientes de kappa
    L_h = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_acoplo = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_val, epsilon_3d, COUPLING_TYPE)
    H_eff = np.kron(L_h, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op) + H_acoplo
    
    # Operadores mixtos
    X_mix = Xh_op - (0.5 * hbar * kappa_val * Lambda_ph_op)
    P_mix = Ph_op + (0.5 * hbar * kappa_val * Lambda_xh_op)
    
    # Operadores en el espacio conjunto
    X_mix_joint = np.kron(X_mix, id_q)
    P_mix_joint = np.kron(P_mix, id_q)
    Commutator_op_joint = np.kron((X_mix @ P_mix) - (P_mix @ X_mix), id_q)
    # Operador para el término <AB + BA>/2 (realmente aquí es lo mismo que A @ B)
    Symmetrized_XmixXq_op = 0.5 * (np.kron(X_mix, X_q_op) + np.kron(np.identity(dim_H_hybrid), X_q_op) @ np.kron(X_mix, id_q))

    for j, t_val in enumerate(time_grid_3d):
        psi_t = joint_evolution_wrapper(psi0_joint_3d, H_eff, t_val, hbar)
        
        # Calcular valores esperados individuales para la covarianza
        exp_X_mix = expected_value(psi_t, X_mix_joint)
        exp_X_q = expected_value(psi_t, Xq_joint_op)
        
        # Calcular el término simetrizado
        exp_symm_XmixXq = expected_value(psi_t, Symmetrized_XmixXq_op)
        
        # Almacenar todos los valores
        results_3d['xc'][i, j] = np.real(expected_value(psi_t, Xh_joint_op))
        results_3d['pc'][i, j] = np.real(expected_value(psi_t, Ph_joint_op))
        results_3d['xq'][i, j] = np.real(exp_X_q)
        results_3d['pq'][i, j] = np.real(expected_value(psi_t, Pq_joint_op))
        results_3d['comm'][i, j] = np.imag(expected_value(psi_t, Commutator_op_joint))
        results_3d['xmix'][i, j] = np.real(exp_X_mix)
        results_3d['pmix'][i, j] = np.real(expected_value(psi_t, P_mix_joint))
        
        # Cálculo de la covarianza
        results_3d['covv'][i, j] = np.real(exp_symm_XmixXq - (exp_X_mix * exp_X_q))


# --- Bucle de Ploteo y Guardado ---
T, K = np.meshgrid(time_grid_3d, kappa_grid_3d)

print("\nGenerando y guardando gráficos 3D...")
for key, label in observables_to_plot_3d.items():
    data_grid = results_3d[key]
    
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot_surface(T, K, data_grid, edgecolor = "royalblue", rstride=1, cstride=1, antialiased=True, alpha = 0.225, lw = 0.1)
    ax.set_ylim(-0.2,1.2)
    ax.contour(T, K, data_grid, zdir='y', offset=1.2, cmap='coolwarm')
    
    ax.set_xlabel('Tiempo (u.a.)', fontsize=12, labelpad=10)
    ax.set_ylabel('Parámetro $\\kappa$', fontsize=12, labelpad=10)
    ax.set_zlabel('Valor', fontsize=12, labelpad=20)
    ax.set_title(f'Evolución del Observable: {label}', fontsize=16)
    
    plt.tight_layout(pad=1.5)
    
    # Guardar la figura con el sufijo solicitado
    base_filename = f"3D_Observable_{key}"
    save_figure_with_timestamp(base_filename, POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches = 'tight', pad_inches = 1)
    
    plt.show()

In [ ]:
# --- Celda de Análisis: Gráfico 3D de Entrelazamiento (CORREGIDA) ---
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# --- Parámetros del Barrido ---
kappa_grid_3d = np.linspace(0, 1.0, 15)
time_grid_3d = np.linspace(0, 4 * np.pi, 100)
epsilon_3d = 0.2

# --- Definición de Observables ---
observables_to_plot_3d = {
    r'$\langle \hat{X}_C \rangle$': np.kron(Xh_op, id_q),
    r'$\langle \hat{P}_C \rangle$': np.kron(Ph_op, id_q),
    r'$\langle \hat{X}_Q \rangle$': np.kron(np.identity(dim_H_hybrid), X_q_op),
    r'$\langle \hat{P}_Q \rangle$': np.kron(np.identity(dim_H_hybrid), P_q_op),
    r'$Im(\langle [\hat{X}_{mix}, \hat{P}_{mix}] \rangle)$': None, # Placeholder
    r'$\langle \hat{X}_{mix} \hat{X}_Q \rangle$': None, # Placeholder
    r'$\langle \hat{X}_{mix} \rangle$': None,
    r'$\langle \hat{P}_{mix} \rangle$': None
}

# --- Almacenamiento de Resultados ---
results_3d = {name: np.zeros((len(kappa_grid_3d), len(time_grid_3d))) for name in observables_to_plot_3d}

# --- Estado Inicial ---
psi0_joint_3d = psi0_joint
# --- Bucle Principal de Cálculo ---
print("Generando superficies de valores esperados...")
for i, kappa_val in enumerate(tqdm(kappa_grid_3d, desc="Barriendo kappa")):
    
    # Construir Hamiltoniano y operadores dependientes de kappa
    L_h = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_acoplo = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_val, epsilon_3d, COUPLING_TYPE)
    H_eff = np.kron(L_h, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op) + H_acoplo
    
    # Operadores mixtos
    X_mix = Xh_op - (0.5 * hbar * kappa_val * Lambda_ph_op)
    P_mix = Ph_op + (0.5 * hbar * kappa_val * Lambda_xh_op)
    
    # *** CORRECCIÓN CLAVE AQUÍ ***
    # El conmutador es un operador puramente clásico (36x36)
    Commutator_op_classical = (X_mix @ P_mix) - (P_mix @ X_mix)
    # Se promueve al espacio conjunto (216x216) usando el producto tensorial con la identidad cuántica
    Commutator_op_joint = np.kron(Commutator_op_classical, id_q)
    
    XmixXq_op = np.kron(X_mix, X_q_op)
    
    for j, t_val in enumerate(time_grid_3d):
        psi_t = joint_evolution_wrapper(psi0_joint_3d, H_eff, t_val, hbar)
        
        for name, op in observables_to_plot_3d.items():
            if name == r'$Im(\langle [\hat{X}_{mix}, \hat{P}_{mix}] \rangle)$':
                # Usar el operador conjunto para el cálculo
                results_3d[name][i, j] = np.imag(expected_value(psi_t, Commutator_op_joint))
            elif name == r'$\langle \hat{X}_{mix} \hat{X}_Q \rangle$':
                results_3d[name][i, j] = np.real(expected_value(psi_t, XmixXq_op))
            elif name == r'$\langle \hat{X}_{mix} \rangle$':
                results_3d[name][i, j] = np.real(expected_value(psi_t, np.kron(X_mix, id_q)))
            elif name == r'$\langle \hat{P}_{mix} \rangle$':
                results_3d[name][i, j] = np.real(expected_value(psi_t, np.kron(P_mix, id_q)))
            else:
                results_3d[name][i, j] = np.real(expected_value(psi_t, op))

# --- Bucle de Ploteo y Guardado ---
T, K = np.meshgrid(time_grid_3d, kappa_grid_3d)


print("Generando y guardando gráficos 3D...")
for name, data_grid in results_3d.items():
    fig = plt.figure(figsize=(12, 8))
    
    ax = fig.add_subplot(111, projection='3d')

    ax.plot_surface(T, K, data_grid, edgecolor = "royalblue", rstride=1, cstride=1, antialiased=True, alpha = 0.225, lw = 0.1)
    ax.set_ylim(-0.2,1.2)
    ax.contour(T, K, data_grid, zdir='y', offset=1.2, cmap='coolwarm')
    ax.set_xlabel('Tiempo (u.a.)', fontsize=12, labelpad=10)
    ax.set_ylabel('Parámetro $\\kappa$', fontsize=12, labelpad=10)
    ax.set_zlabel('Valor Esperado', fontsize=12, labelpad=15)
    ax.set_title(f'Evolución del Observable: {name}', fontsize=16)
    
    base_filename = f"3D_Observable_"

    save_figure_with_timestamp(base_filename, POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches = 'tight', pad_inches = 1)
    plt.show()

In [ ]:
# --- ENTRELAZAMIENTO ---
kappa_grid_3d = np.linspace(0, 1.0, 21)
time_grid_3d = np.linspace(0, 4 * np.pi, 100)
epsilon_ent_3d = 0.25
purity_grid = np.zeros((len(kappa_grid_3d), len(time_grid_3d)))

print("Calculando superficie de entrelazamiento...")

for i, kappa_val in enumerate(tqdm(kappa_grid_3d, desc="Barriendo kappa para 3D")):
    L_h = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_acoplo = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_val, epsilon_ent_3d, COUPLING_TYPE)
    H_eff = np.kron(L_h, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op) + H_acoplo
    for j, t in enumerate(time_grid_3d):
        psi_t = joint_evolution_wrapper(psi0_joint, H_eff, t, hbar)
        rho_total = np.outer(psi_t, np.conjugate(psi_t))
        rho_total_reshaped = rho_total.reshape((dim_H_hybrid, dim_fock_q, dim_H_hybrid, dim_fock_q))
        reduced_rho_c = np.einsum('ikjk->ij', rho_total_reshaped)
        purity_grid[i, j] = np.real(np.trace(reduced_rho_c @ reduced_rho_c))

T, K = np.meshgrid(time_grid_3d, kappa_grid_3d)
entanglement_measure = 1 - purity_grid

fig_3d = plt.figure(figsize=(12, 8)); ax_3d = fig_3d.add_subplot(111, projection='3d')
ax_3d.invert_xaxis()
surf = ax_3d.plot_surface(T, K, entanglement_measure, cmap=cm.viridis, rstride=1, cstride=1, antialiased=True, edgecolors=(0, 0, 0, 0.1), lw = 0.1)
ax_3d.set_xlabel('Tiempo (u.a.)', labelpad = 10); ax_3d.set_ylabel('Parámetro $\\kappa$', labelpad = 10)
ax_3d.set_zlabel('Entrelazamiento ($1 - \\mathrm{Tr}(\\rho_C^2)$)', labelpad = 10)
ax_3d.set_title(f'Evolución del Entrelazamiento Clásico-Cuántico ($\\varepsilon={epsilon_ent_3d}$)')
fig_3d.colorbar(surf, shrink=0.5, aspect=10, pad=0.1)
save_figure_with_timestamp("classical_purity_vs_kappa", POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches='tight')
plt.show()

In [ ]:
# --- PUREZA DEL SUBSISTEMA CUÁNTICO VS KAPPA ---
kappa_values_purity = np.logspace(0, -4, 20)
kappa_values_purity = np.append(kappa_values_purity, 0)
time_to_measure = 5.0
purity_q_vs_kappa = []

print(f"Calculando pureza cuántica en t={time_to_measure}s vs kappa...")
for kappa_val in tqdm(kappa_values_purity, desc="Calculando Pureza Cuántica"):
    L_h = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_acoplo = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_val, epsilon, COUPLING_TYPE)
    H_eff = np.kron(L_h, id_q) + np.kron(np.identity(dim_H_hybrid), H_q_op) + H_acoplo
    
    psi_t = joint_evolution_wrapper(psi0_joint, H_eff, time_to_measure, hbar)
    
    # Calcular la matriz densidad reducida cuántica
    rho_total = np.outer(psi_t, np.conjugate(psi_t))
    rho_total_reshaped = rho_total.reshape((dim_H_hybrid, dim_fock_q, dim_H_hybrid, dim_fock_q))
    
    rho_q_reduced = np.einsum('jijl->il', rho_total_reshaped)
    
    # Calcular la pureza
    purity = 1 - np.real(np.trace(rho_q_reduced @ rho_q_reduced))
    purity_q_vs_kappa.append(purity)

# Graficar
fig_purity, ax_purity = plt.subplots(figsize=(10, 7))
ax_purity.plot(kappa_values_purity, purity_q_vs_kappa, 'o-')
ax_purity.set_title(f'Entrelazamiento cuántico $1 -$ Tr($\\rho_Q^2$) en $t={time_to_measure}$ en función de $\\kappa$', pad = 10)
ax_purity.set_xlabel('$\\kappa$')
ax_purity.set_ylabel('Entrelazamiento ($1 - \\mathrm{Tr}(\\rho_C^2)$)')
ax_purity.set_xscale('log')
ax_purity.invert_xaxis()
add_grid(ax_purity)
save_figure_with_timestamp("quantum_purity_vs_kappa", POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches='tight')
plt.show()

# 6. Análisis de truncamiento

In [ ]:
# --- VALIDEZ DEL CONMUTADOR ---

Nh_grid_comm = np.array([3, 4, 5, 6, 7, 8])
time_grid_comm = np.linspace(0, 4 * np.pi, 100)
kappa_comm_test = 1.0
epsilon_comm_test = 0.1

commutator_surface = np.zeros((len(Nh_grid_comm), len(time_grid_comm)))

def setup_system_for_Nh_comm(N_h, N_q, kappa_val, epsilon_val, coupling_type_val):
    dim_fock_h = N_h + 1; dim_H = dim_fock_h ** 2; dim_q = N_q + 1
    id_h_s = np.identity(dim_fock_h); a_h_s = np.diag([np.sqrt(i) for i in range(1, dim_fock_h)], k=1); aPlus_h_s = a_h_s.T
    a_x = np.kron(a_h_s, id_h_s); aPlus_x = np.kron(aPlus_h_s, id_h_s)
    a_p = np.kron(id_h_s, a_h_s); aPlus_p = np.kron(id_h_s, aPlus_h_s)
    Xh = np.sqrt(hbar/(2*m_h*omega_h)) * (aPlus_x + a_x); Lxh = 1j * np.sqrt((hbar*m_h*omega_h)/2) * (aPlus_x - a_x)
    Ph = np.sqrt(hbar*m_h*omega_h/2) * (aPlus_p + a_p); Lph = 1j * np.sqrt((hbar*m_h*omega_h)/2) * (aPlus_p - a_p)
    id_q_N = np.identity(dim_q); a_q_N = np.diag([np.sqrt(i) for i in range(1, dim_q)], k=1); aPlus_q_N = a_q_N.T
    Xq = np.sqrt(hbar/(2*m_q*omega_q)) * (aPlus_q_N + a_q_N); Pq = 1j * np.sqrt(hbar*m_q*omega_q/2) * (aPlus_q_N - a_q_N)
    Hq = hbar * omega_q * (aPlus_q_N @ a_q_N + 0.5 * id_q_N)
    L_h = L_h_generator(Xh, Ph, Lxh, Lph, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_acoplo = create_unified_coupling_term(Xh, Ph, Lxh, Lph, Xq, Pq, hbar, kappa_val, epsilon_val, coupling_type_val)
    H_eff = np.kron(L_h, id_q_N) + np.kron(np.identity(dim_H), Hq) + H_acoplo
    X_mix = Xh - (0.5 * hbar * kappa_val * Lph)
    P_mix = Ph + (0.5 * hbar * kappa_val * Lxh)
    Commutator_op_classical = (X_mix @ P_mix) - (P_mix @ X_mix)
    Commutator_op_joint = np.kron(Commutator_op_classical, id_q_N)
    return H_eff, Commutator_op_joint, dim_H, dim_q

print("Generando superficie de validez del conmutador...")
for i, N_h in enumerate(tqdm(Nh_grid_comm, desc="Barriendo N_max_h")):
    H_eff, Commutator_op_joint, dim_h, dim_q = setup_system_for_Nh_comm(N_h, N_max_q, kappa_comm_test, epsilon_comm_test, COUPLING_TYPE)
    
    # --- Creación del estado inicial consistente para ESTA dimensión ---
    # Usamos las mismas definiciones que en la celda de configuración inicial
    x0_c = 1.5; p0_c = 0.5
    alpha_x_cat = x0_c / np.sqrt(2.0); alpha_p_cat = p0_c / np.sqrt(2.0)
    
    psi0_c_x_coeffs_temp = twin_coherent_state_coeffs(alpha_x_cat, N_h)
    psi0_c_p_coeffs_temp = coherent_state_coeffs(alpha_p_cat, N_h)
    psi0_c_coeffs_temp = normalize(np.kron(psi0_c_x_coeffs_temp, psi0_c_p_coeffs_temp))
    psi0_c_coeffs_temp[0] /= 2
    psi0_c_coeffs_temp = normalize(psi0_c_coeffs_temp)
    
    n0_q = 2.0; sigma_q = 1.5
    psi0_q_coeffs_temp = gaussian_fock_state_coeffs(n0_q, sigma_q, N_max_q)
    
    psi0_joint_temp = normalize(np.kron(psi0_c_coeffs_temp, psi0_q_coeffs_temp))
    
    for j, t_val in enumerate(time_grid_comm):
        psi_t = joint_evolution_wrapper(psi0_joint_temp, H_eff, t_val)
        commutator_surface[i, j] = np.imag(expected_value(psi_t, Commutator_op_joint))

T_comm, N_comm = np.meshgrid(time_grid_comm, Nh_grid_comm)
fig_comm_3d = plt.figure(figsize=(12, 8))
fig_comm_3d = plt.figure(figsize=(12, 8))
ax_comm_3d = fig_comm_3d.add_subplot(111, projection='3d')
surf = ax_comm_3d.plot_surface(T_comm, N_comm, commutator_surface, cmap=cm.viridis, rstride=1, cstride=1, antialiased=True, edgecolors=(0, 0, 0, 0.1), lw = 0.1)
ax_comm_3d.plot(time_grid_comm, np.full_like(time_grid_comm, Nh_grid_comm.max() + 0.5), hbar * kappa_comm_test, color='black', linestyle='--', label=f'Valor Teórico ({hbar * kappa_comm_test:.2f})')

ax_comm_3d.set_xlabel('Tiempo (u.a.)', fontsize=12, labelpad=10)
ax_comm_3d.set_ylabel('$N_{max,h}$', fontsize=12, labelpad=10)
ax_comm_3d.set_zlabel(r'$Im(\langle [\hat{X}_{mix}, \hat{P}_{mix}] \rangle)$', fontsize=12, labelpad=20)
ax_comm_3d.set_title(f'Validez del Conmutador vs. Tamaño de Base y Tiempo ($\\kappa={kappa_comm_test}$)', fontsize=16)
fig_comm_3d.colorbar(surf, shrink=0.5, aspect=10, pad=0.1)
ax_comm_3d.legend()

save_figure_with_timestamp("commutator_validity_3D", POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches='tight')
plt.show()

In [ ]:
# --- CONVERGENCIA DEL BACK-REACTION vs. TAMAÑO DE BASE ---

def setup_system_for_Nh_br_2d(N_h, N_q, kappa_val, epsilon_val, coupling_type_val):
    """Configura el sistema para un N_max_h específico para el test de retro-reacción."""

    dim_fock_h = N_h + 1; dim_H = dim_fock_h ** 2; dim_q = N_q + 1
    id_h_s = np.identity(dim_fock_h); a_h_s = np.diag([np.sqrt(i) for i in range(1, dim_fock_h)], k=1); aPlus_h_s = a_h_s.T
    a_x = np.kron(a_h_s, id_h_s); aPlus_x = np.kron(aPlus_h_s, id_h_s)
    a_p = np.kron(id_h_s, a_h_s); aPlus_p = np.kron(id_h_s, aPlus_h_s)
    Xh = np.sqrt(hbar/(2*m_h*omega_h)) * (aPlus_x + a_x); Lxh = 1j * np.sqrt((hbar*m_h*omega_h)/2) * (aPlus_x - a_x)
    Ph = np.sqrt(hbar*m_h*omega_h/2) * (aPlus_p + a_p); Lph = 1j * np.sqrt((hbar*m_h*omega_h)/2) * (aPlus_p - a_p)
    id_q_N = np.identity(dim_q); a_q_N = np.diag([np.sqrt(i) for i in range(1, dim_q)], k=1); aPlus_q_N = a_q_N.T
    Xq = np.sqrt(hbar/(2*m_q*omega_q)) * (aPlus_q_N + a_q_N); Pq = 1j * np.sqrt(hbar*m_q*omega_q/2) * (aPlus_q_N - a_q_N)
    Hq = hbar * omega_q * (aPlus_q_N @ a_q_N + 0.5 * id_q_N)
    L_h = L_h_generator(Xh, Ph, Lxh, Lph, m_h, omega_h, hbar, kappa_val, POTENTIAL_TYPE, C4)
    H_acoplo = create_unified_coupling_term(Xh, Ph, Lxh, Lph, Xq, Pq, hbar, kappa_val, epsilon_val, coupling_type_val)
    H_eff = np.kron(L_h, id_q_N) + np.kron(np.identity(dim_H), Hq) + H_acoplo
    Xc_joint = np.kron(Xh, id_q_N)
    return H_eff, Xc_joint, dim_H, dim_q

# --- CELDA DE ANÁLISIS: CONVERGENCIA DE LA RETRO-REACCIÓN vs. TAMAÑO DE BASE (MÉTRICAS ROBUSTAS) ---

# --- Parámetros del Experimento ---
Nh_values_br = [3, 4, 5, 6, 7, 8]  # Valores para N_max_h a probar
kappa_br_test = 0.0              # Siempre kappa=0 para analizar la retro-reacción en KvN
epsilon_br_test = 0.1
times_br = np.linspace(0, 10 * np.pi, 200) # Rango de tiempo para el cálculo

# Listas para almacenar las métricas finales
backreaction_mae_list = []
backreaction_rmse_list = []

print(f"Probando convergencia de back-reaction para kappa = {kappa_br_test}...")
for N_h in tqdm(Nh_values_br, desc="Variando tamaño de base N_max_h"):
    
    # 1. Configurar el sistema para el N_h y kappa actuales
    H_eff, Xc_joint, dim_h, dim_q = setup_system_for_Nh_br_2d(N_h, N_max_q, kappa_br_test, epsilon_br_test, COUPLING_TYPE)
    
    # 2. Definir estados iniciales ESTRUCTURADOS y consistentes
    # Estado Clásico: Un estado coherente fijo para todas las simulaciones.
    # Los parámetros alpha definen la posición y momento inicial del paquete de onda.
    alpha_x_c = 1.0 / np.sqrt(2)
    alpha_p_c = 0.5 / np.sqrt(2)
    psi0_c_x_coeffs = coherent_state_coeffs(alpha_x_c, N_h)
    psi0_c_p_coeffs = coherent_state_coeffs(alpha_p_c, N_h)
    psi0_c_temp = normalize(np.kron(psi0_c_x_coeffs, psi0_c_p_coeffs))

    # Estados Cuánticos: Dos estados gaussianos distintos pero con estructura similar.
    # El primero centrado en n=1, el segundo en n=2.
    psi0_q1 = gaussian_fock_state_coeffs(n0=1, sigma=1.0, N_max=N_max_q)
    psi0_q2 = gaussian_fock_state_coeffs(n0=2, sigma=1.0, N_max=N_max_q)
    
    # Estados Híbridos
    psi0_h1 = np.kron(psi0_c_temp, psi0_q1)
    psi0_h2 = np.kron(psi0_c_temp, psi0_q2)
    
    # 3. Evolucionar ambos sistemas y obtener las trayectorias de <Xc>
    ex_xc_1_traj = [np.real(expected_value(joint_evolution_wrapper(psi0_h1, H_eff, t_val), Xc_joint)) for t_val in times_br]
    ex_xc_2_traj = [np.real(expected_value(joint_evolution_wrapper(psi0_h2, H_eff, t_val), Xc_joint)) for t_val in times_br]
        
    # 4. Calcular y guardar las métricas de back-reaction para esta N_h
    difference_vector = np.array(ex_xc_1_traj) - np.array(ex_xc_2_traj)
    
    # Métrica 1: Promedio de la Diferencia Absoluta (MAE) sobre toda la trayectoria
    mae = np.mean(np.abs(difference_vector))
    backreaction_mae_list.append(mae)
    
    # Métrica 2: Raíz del Error Cuadrático Medio (RMSE) sobre toda la trayectoria
    rmse = np.sqrt(np.mean(np.square(difference_vector)))
    backreaction_rmse_list.append(rmse)
    
    print(f"  N_max_h = {N_h}: MAE = {mae:.2e}, RMSE = {rmse:.2e}")

fig_br_conv, ax_br_conv = plt.subplots(figsize=(10, 7))

ax_br_conv.plot(Nh_values_br, backreaction_mae_list, 'o-', label='MAE (Promedio Absoluto)', color='C0', markersize=8)
ax_br_conv.plot(Nh_values_br, backreaction_rmse_list, 's--', label='RMSE (Raíz Cuadrática Media)', color='C1', markersize=8)

ax_br_conv.set_title('Convergencia del Artefacto de Retro-reacción vs. Tamaño de Base')
ax_br_conv.set_xlabel('$N_{max,h}$ (Dimensión de la base para $x_c$ y $p_c$)')
ax_br_conv.set_ylabel('Magnitud de la Retro-reacción (u.a.)')
ax_br_conv.set_xticks(Nh_values_br)
ax_br_conv.set_yscale('log')
ax_br_conv.legend()
add_grid(ax_br_conv)

save_figure_with_timestamp("backreaction_convergence_vs_N_2D", POTENTIAL_TYPE, COUPLING_TYPE, bbox_inches='tight')
plt.show()

# 7. Visualizaciones Dinámicas

In [ ]:
# --- DENSIDADES DE PROBABILIDAD MARGINALES (Fock) ---

# Pre-cálculo de evoluciones
times_anim_marginal = np.linspace(0, 4 * np.pi, 150)
evolved_psis_no_acoplo = [joint_evolution_wrapper(psi0_joint, H_eff_sin_acoplo_default, t) for t in tqdm(times_anim_marginal, desc="Pre-calculando Sin Acoplo")]
evolved_psis_con_acoplo = [joint_evolution_wrapper(psi0_joint, H_eff_con_acoplo_default, t) for t in tqdm(times_anim_marginal, desc="Pre-calculando Con Acoplo")]

fig_joint_anim = plt.figure(figsize=(12, 10))
gs = fig_joint_anim.add_gridspec(2, 2)
ax_c_no_acoplo_anim = fig_joint_anim.add_subplot(gs[0, 0], projection='3d')
ax_q_no_acoplo_anim = fig_joint_anim.add_subplot(gs[0, 1])
ax_c_con_acoplo_anim = fig_joint_anim.add_subplot(gs[1, 0], projection='3d')
ax_q_con_acoplo_anim = fig_joint_anim.add_subplot(gs[1, 1])

# Grid para el subsistema clásico
_n_indices_h = np.arange(dim_fock_h_single) # Indices para nx_c y np_c
_nxc_grid, _npc_grid = np.meshgrid(_n_indices_h, _n_indices_h, indexing='xy') # Usar 'xy' es más estándar para meshgrid con bar3d si se usa .T.ravel()
nxc_pos_flat, npc_pos_flat = _nxc_grid.ravel(), _npc_grid.ravel() # Estos serán los x, y para bar3d
bottom_h = np.zeros_like(nxc_pos_flat, dtype=float)

# Ticks para el subsistema cuántico
_n_indices_q = np.arange(dim_fock_q)
tick_labels_q = [rf'$|{int(i)}\rangle$' for i in _n_indices_q]
tick_labels_h_anim = [rf'$|{int(i)}\rangle$' for i in _n_indices_h]

bar_width_anim = 0.35
bar_depth_anim = 0.35
max_prob_c_overall_anim = 0.01
max_prob_q_overall_anim = 0.01

for i in range(len(times_anim_marginal)):
    # Sin acoplo
    prob_total_no_acoplo_matrix_form = prob_density_coeffs(evolved_psis_no_acoplo[i]).reshape((dim_fock_h_single, dim_fock_h_single, dim_fock_q))
    prob_c_marginal_no_acoplo = np.sum(prob_total_no_acoplo_matrix_form, axis=2) # Sumar sobre el eje cuántico (el último)
    prob_q_marginal_no_acoplo = np.sum(prob_total_no_acoplo_matrix_form, axis=(0,1)) # Sumar sobre los dos ejes clásicos
    
    max_prob_c_overall_anim = max(max_prob_c_overall_anim, np.max(prob_c_marginal_no_acoplo))
    max_prob_q_overall_anim = max(max_prob_q_overall_anim, np.max(prob_q_marginal_no_acoplo))
    
    # Con acoplo
    prob_total_con_acoplo_matrix_form = prob_density_coeffs(evolved_psis_con_acoplo[i]).reshape((dim_fock_h_single, dim_fock_h_single, dim_fock_q))
    prob_c_marginal_con_acoplo = np.sum(prob_total_con_acoplo_matrix_form, axis=2)
    prob_q_marginal_con_acoplo = np.sum(prob_total_con_acoplo_matrix_form, axis=(0,1))

    max_prob_c_overall_anim = max(max_prob_c_overall_anim, np.max(prob_c_marginal_con_acoplo))
    max_prob_q_overall_anim = max(max_prob_q_overall_anim, np.max(prob_q_marginal_con_acoplo))

if max_prob_c_overall_anim < 1e-9: max_prob_c_overall_anim = 0.1 # Evitar límite cero si todo es cero
if max_prob_q_overall_anim < 1e-9: max_prob_q_overall_anim = 0.1


def update_joint_density_plot(frame_num):
    t = times_anim_marginal[frame_num]
    
    psi_t_total_no_acoplo = evolved_psis_no_acoplo[frame_num]
    psi_t_total_con_acoplo = evolved_psis_con_acoplo[frame_num]

    # --- SIN ACOPLO ---
    # Psi_total tiene forma ( (N_max_c+1)^2 * (N_max_q+1), )
    # Lo reformamos a ((N_max_c+1), (N_max_c+1), (N_max_q+1)) para sumar sobre ejes
    prob_total_no_acoplo_reshaped = prob_density_coeffs(psi_t_total_no_acoplo).reshape((dim_fock_h_single, dim_fock_h_single, dim_fock_q))
    
    # Densidad marginal clásica:
    prob_c_marginal_no_acoplo = np.sum(prob_total_no_acoplo_reshaped, axis=2)
    
    ax_c_no_acoplo_anim.cla()
    ax_c_no_acoplo_anim.bar3d(nxc_pos_flat, npc_pos_flat, bottom_h, bar_width_anim, bar_depth_anim,
                               np.array(prob_c_marginal_no_acoplo.ravel(), dtype=float),
                               color='red', shade=True)
    ax_c_no_acoplo_anim.set_title(f'Sin Acoplo: $\\rho_C(n_x, n_p)$ a $t={t:.2f}$')
    ax_c_no_acoplo_anim.set_xlabel('$n_x$', labelpad = 10); ax_c_no_acoplo_anim.set_ylabel('$n_p$', labelpad = 10)
    ax_c_no_acoplo_anim.set_xticks(_n_indices_h); ax_c_no_acoplo_anim.set_xticklabels(tick_labels_h_anim)
    ax_c_no_acoplo_anim.set_yticks(_n_indices_h); ax_c_no_acoplo_anim.set_yticklabels(tick_labels_h_anim)
    ax_c_no_acoplo_anim.invert_xaxis(); ax_c_no_acoplo_anim.set_zlim(0, max_prob_c_overall_anim)

    # Densidad marginal cuántica:
    prob_q_marginal_no_acoplo = np.sum(prob_total_no_acoplo_reshaped, axis=(0,1))
    ax_q_no_acoplo_anim.cla()
    ax_q_no_acoplo_anim.bar(_n_indices_q, np.array(prob_q_marginal_no_acoplo, dtype=float), color='red', width=0.8)
    ax_q_no_acoplo_anim.set_title(f'Sin Acoplo: $\\rho_Q(n_q)$ a $t={t:.2f}$')
    ax_q_no_acoplo_anim.set_xlabel('$n_q$'); ax_q_no_acoplo_anim.set_ylabel('$|c|^2$')
    ax_q_no_acoplo_anim.set_xticks(_n_indices_q); ax_q_no_acoplo_anim.set_xticklabels(tick_labels_q)
    ax_q_no_acoplo_anim.set_ylim(0, max_prob_q_overall_anim)

    # --- CON ACOPLO ---
    prob_total_con_acoplo_reshaped = prob_density_coeffs(psi_t_total_con_acoplo).reshape((dim_fock_h_single, dim_fock_h_single, dim_fock_q))
    prob_c_marginal_con_acoplo = np.sum(prob_total_con_acoplo_reshaped, axis=2)

    ax_c_con_acoplo_anim.cla()
    ax_c_con_acoplo_anim.bar3d(nxc_pos_flat, npc_pos_flat, bottom_h, bar_width_anim, bar_depth_anim,
                                np.array(prob_c_marginal_con_acoplo.ravel(), dtype=float),
                                color='navy', shade=True)
    ax_c_con_acoplo_anim.set_title(f'Con Acoplo ($\\varepsilon={epsilon}$): $\\rho_C(n_x, n_p)$ a $t={t:.2f}$')
    ax_c_con_acoplo_anim.set_xlabel('$n_x$', labelpad = 10); ax_c_con_acoplo_anim.set_ylabel('$n_p$', labelpad = 10)
    ax_c_con_acoplo_anim.set_xticks(_n_indices_h); ax_c_con_acoplo_anim.set_xticklabels(tick_labels_h_anim)
    ax_c_con_acoplo_anim.set_yticks(_n_indices_h); ax_c_con_acoplo_anim.set_yticklabels(tick_labels_h_anim)
    ax_c_con_acoplo_anim.invert_xaxis(); ax_c_con_acoplo_anim.set_zlim(0, max_prob_c_overall_anim)
    
    prob_q_marginal_con_acoplo = np.sum(prob_total_con_acoplo_reshaped, axis=(0,1))
    ax_q_con_acoplo_anim.cla()
    ax_q_con_acoplo_anim.bar(_n_indices_q, np.array(prob_q_marginal_con_acoplo, dtype=float), color='navy', width=0.8)
    ax_q_con_acoplo_anim.set_title(f'Con Acoplo ($\\varepsilon={epsilon}$): $\\rho_Q(n_q)$ a $t={t:.2f}$')
    ax_q_con_acoplo_anim.set_xlabel('$n_q$'); ax_q_con_acoplo_anim.set_ylabel('$|c|^2$')
    ax_q_con_acoplo_anim.set_xticks(_n_indices_q); ax_q_con_acoplo_anim.set_xticklabels(tick_labels_q)
    ax_q_con_acoplo_anim.set_ylim(0, max_prob_q_overall_anim)
    
    fig_joint_anim.suptitle(f'Densidades de Probabilidad Marginales Híbridas a $t={t:.2f}$s', fontsize=16)
    return fig_joint_anim,

# Crear la animación comparativa
ani_joint_density = FuncAnimation(fig_joint_anim, update_joint_density_plot, frames=len(times_anim_marginal), interval=100, blit = False)

In [ ]:
# --- DENSIDADES DE PROBABILIDAD MARGINALES (Espacio de fases) ---

# --- Interruptor de Control para la Reconstrucción Clásica ---
# True: Usa la función Q de Husimi (precisa, lenta).
# False: Usa la proyección de la función de onda (aproximada, rápida).
USE_PRECISE_CLASSICAL_RECONSTRUCTION = True 

# --- Parámetros de Animación ---
kappa_anim = 1.0
epsilon_anim = 0.2
times_anim = np.linspace(0, 4 * np.pi, 150)

# --- Construir Hamiltonianos para la animación (Con y Sin Acoplo) ---
L_h_anim = L_h_generator(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, m_h, omega_h, hbar, kappa_anim, "quartic", 0.1)
H_q_op_full = np.kron(np.identity(dim_H_hybrid), H_q_op)
L_h_op_full = np.kron(L_h_anim, id_q)

H_eff_anim_no_coupling = L_h_op_full + H_q_op_full
H_acoplo_anim = create_unified_coupling_term(Xh_op, Ph_op, Lambda_xh_op, Lambda_ph_op, X_q_op, P_q_op, hbar, kappa_anim, epsilon_anim, 'x_mix_x')
H_eff_anim_with_coupling = H_eff_anim_no_coupling + H_acoplo_anim

# --- Pre-cálculo de estados evolucionados ---
evolved_psis_no_coupling = [joint_evolution_wrapper(psi0_joint, H_eff_anim_no_coupling, t) for t in tqdm(times_anim, desc="Pre-calculando Sin Acoplo")]
evolved_psis_with_coupling = [joint_evolution_wrapper(psi0_joint, H_eff_anim_with_coupling, t) for t in tqdm(times_anim, desc="Pre-calculando Con Acoplo")]

# --- Grids y bases de funciones de onda ---
x_range = np.linspace(-5, 5, 80)
p_range = np.linspace(-5, 5, 80)
X_grid, P_grid = np.meshgrid(x_range, p_range)
wavefunctions_basis_x = np.array([number_state_wavefunction(n, x_range, m_h, omega_h, hbar) for n in range(dim_fock_h_single)])
wavefunctions_basis_p = np.array([number_state_wavefunction(n, p_range, m_h, omega_h, hbar) for n in range(dim_fock_h_single)])
wavefunctions_basis_q = np.array([number_state_wavefunction(n, x_range, m_q, omega_q, hbar) for n in range(dim_fock_q)])

# --- Configuración de la Figura ---
fig_anim = plt.figure(figsize=(16, 14), constrained_layout=True)
gs = fig_anim.add_gridspec(2, 2)
ax_phase_no_c = fig_anim.add_subplot(gs[0, 0], projection='3d')
ax_pos_q_no_c = fig_anim.add_subplot(gs[0, 1])
ax_phase_with_c = fig_anim.add_subplot(gs[1, 0], projection='3d')
ax_pos_q_with_c = fig_anim.add_subplot(gs[1, 1])
fig_anim.suptitle(f'Evolución del Sistema Híbrido ($\\kappa={kappa_anim}$)', fontsize=18)

# --- Cálculo de máximos globales para escalas de ejes ---
max_z_phase = 0.01
for i in range(len(times_anim)):
    # Sin acoplo
    psi_t_no_c = evolved_psis_no_coupling[i]
    psi_c_vec_no_c = np.sum(psi_t_no_c.reshape(dim_H_hybrid, dim_fock_q), axis=1)
    psi_c_mat_no_c = psi_c_vec_no_c.reshape(dim_fock_h_single, dim_fock_h_single)
    psi_xp_no_c = np.einsum('nm,nx,mp->xp', psi_c_mat_no_c, wavefunctions_basis_x, wavefunctions_basis_p)
    max_z_phase = max(max_z_phase, np.max(np.abs(psi_xp_no_c)**2))
    
    # Con acoplo
    psi_t_with_c = evolved_psis_with_coupling[i]
    psi_c_vec_with_c = np.sum(psi_t_with_c.reshape(dim_H_hybrid, dim_fock_q), axis=1)
    psi_c_mat_with_c = psi_c_vec_with_c.reshape(dim_fock_h_single, dim_fock_h_single)
    psi_xp_with_c = np.einsum('nm,nx,mp->xp', psi_c_mat_with_c, wavefunctions_basis_x, wavefunctions_basis_p)
    max_z_phase = max(max_z_phase, np.max(np.abs(psi_xp_with_c)**2))

def update_animation_comparative(frame):
    t = times_anim[frame]
    
    cases = [
        (evolved_psis_no_coupling[frame], ax_phase_no_c, ax_pos_q_no_c, 'Sin Acoplo', 'viridis', 'magenta'),
        (evolved_psis_with_coupling[frame], ax_phase_with_c, ax_pos_q_with_c, f'Con Acoplo ($\\varepsilon={epsilon_anim}$)', 'plasma', 'cyan')
    ]

    for psi_t, ax_phase, ax_pos_q, title_prefix, cmap, q_color in cases:
        ax_phase.cla()
        ax_pos_q.cla()
        
        # --- Subsistema Clásico ---
        rho_total = np.outer(psi_t, np.conjugate(psi_t))
        rho_reshaped = rho_total.reshape((dim_H_hybrid, dim_fock_q, dim_H_hybrid, dim_fock_q))
        
        if USE_PRECISE_CLASSICAL_RECONSTRUCTION:
            # --- MÉTODO PRECISO Y LENTO ---
            rho_c_matrix = np.trace(rho_reshaped, axis1=1, axis2=3)
            prob_dens_xp = reconstruct_husimi_from_rho_c(rho_c_matrix, x_range, p_range, N_max_h)
            z_label = r'$Q(x_c, p_c)$'
        else:
            # --- MÉTODO APROXIMADO Y RÁPIDO ---
            psi_c_vec = np.sum(psi_t.reshape(dim_H_hybrid, dim_fock_q), axis=1)
            psi_c_mat = psi_c_vec.reshape(dim_fock_h_single, dim_fock_h_single)
            psi_xp = np.einsum('nm,nx,mp->xp', psi_c_mat, wavefunctions_basis_x, wavefunctions_basis_p)
            prob_dens_xp = np.abs(psi_xp)**2
            
            rho_c_matrix = np.trace(rho_reshaped, axis1=1, axis2=3)
            z_label = r'Visualización de $\rho(x_c, p_c)$'

        ax_phase.plot_surface(X_grid, P_grid, prob_dens_xp.T, cmap=cmap)
        ax_phase.set_xlabel('$x_c$', labelpad = 10); ax_phase.set_ylabel('$p_c$', labelpad = 10); ax_phase.set_zlabel(z_label, labelpad = 20)
        ax_phase.set_title(f'{title_prefix}: Clásico $t={t:.2f}$')

        # --- Subsistema Cuántico ---
        rho_q = np.trace(rho_reshaped, axis1=0, axis2=2)
        prob_dens_q = np.real(np.einsum('nm,nx,mx->x', rho_q, wavefunctions_basis_q.conj(), wavefunctions_basis_q))
        
        ax_pos_q.plot(x_range, prob_dens_q, color=q_color, lw=2)
        ax_pos_q.set_ylim(0, 1.0); ax_pos_q.set_xlim(x_range[0], x_range[-1])
        ax_pos_q.set_title(f'{title_prefix}: Cuántico $t={t:.2f}$')
        ax_pos_q.set_xlabel('$x_q$'); ax_pos_q.set_ylabel(r'$\rho(x_q, t)$')
        add_grid(ax_pos_q)

    return fig_anim,

ani_comp = FuncAnimation(fig_anim, update_animation_comparative, frames=len(times_anim), interval=100, blit=False)

In [ ]:
# Guardar la animación de densidades marginales
save_animation_with_timestamp(ani_joint_density, "joint_evolution", POTENTIAL_TYPE, COUPLING_TYPE, dpi=150, writer='pillow')

In [ ]:
# Guardar la animación comparativa del espacio de fases
save_animation_with_timestamp(ani_comp, "phase_space_comparison", POTENTIAL_TYPE, COUPLING_TYPE, dpi=150, writer='pillow')